# [Exploration 19] KorQuAD_BERT : 인간보다 퀴즈를 잘푸는 인공지능 

pretrained BERT모델을 이용해 KorQuAD task를 수행하는 프로젝트

---

## 프로젝트 목표
---
- pretrained BERT의 성능을 BERT와 비교한다.
- KorQuAD 데이터셋에 대한 전처리 과정을 공부한다. 
- QuAD task 해결과정을 공부한다. 


## 프로젝트 설명
---
0. 데이터 불러오기, 전처리
1. pretrained model 로딩하기
2. pretrained model finetune 하기
3. Inference 수행하기
4. 학습 경과 시각화 비교분석

## 0. 데이터 불러오기 및 전처리

In [1]:
# imports
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow_addons as tfa

import os
import re
import numpy as np
import pandas as pd
import pickle
import random
import collections
import json
from datetime import datetime

import sentencepiece as spm
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

random_seed = 2022
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

In [2]:
def print_json_tree(data, indent=""):
    for key, value in data.items(): # items() : key, value 한번에 가져오기
        if type(value) == list:
            print(f'{indent}-{key}:[{len(value)}]')
            print_json_tree(value[0], indent+" ") # 첫번째 아이템 출력
        else:
            print(f'{indent}-{key}:{value}')

In [3]:
data_dir = os.getenv('HOME')+'/aiffel/bert_qna/data'
model_dir = os.getenv('HOME')+'/aiffel/bert_qna/models'

- 실제 데이터의 형태

In [7]:
print(json.dumps(train_json['data'][0], indent=2, ensure_ascii=False))

{
  "paragraphs": [
    {
      "qas": [
        {
          "answers": [
            {
              "text": "교향곡",
              "answer_start": 54
            }
          ],
          "id": "6566495-0-0",
          "question": "바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?"
        },
        {
          "answers": [
            {
              "text": "1악장",
              "answer_start": 421
            }
          ],
          "id": "6566495-0-1",
          "question": "바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?"
        },
        {
          "answers": [
            {
              "text": "베토벤의 교향곡 9번",
              "answer_start": 194
            }
          ],
          "id": "6566495-0-2",
          "question": "바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?"
        },
        {
          "answers": [
            {
              "text": "파우스트",
              "answer_start": 15
            }
          ],
          "id": "6566518-0-0",
          "question": "1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?"
        },
        {
    

- **print_json_tree함수**를 이용해 리스트의 **첫번째 아이템**의 내용을 확인합니다.

In [5]:
train_json_path = data_dir + '/KorQuAD_v1.0_train.json'
with open(train_json_path) as f:
    train_json = json.load(f)
    print_json_tree(train_json)

-version:KorQuAD_v1.0_train
-data:[1420]
 -paragraphs:[3]
  -qas:[8]
   -answers:[1]
    -text:교향곡
    -answer_start:54
   -id:6566495-0-0
   -question:바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?
  -context:1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.
 -title:파우스트_서곡


In [6]:
dev_json_path = data_dir + '/KorQuAD_v1.0_dev.json'
with open(dev_json_path) as f:
    dev_json = json.load(f)
    print_json_tree(dev_json)

-version:KorQuAD_v1.0_dev
-data:[140]
 -paragraphs:[2]
  -qas:[7]
   -answers:[1]
    -text:1989년 2월 15일
    -answer_start:0
   -id:6548850-0-0
   -question:임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?
  -context:1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률위반)으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일~20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다.
 -title:임종석


## 0-1. 전처리 - 띄어쓰기 단위 정보관리

In [8]:
def _is_whitespace(c):
    if c == ' ' or c == '\t' or c == '\r' or c == '\n' or ord(c) == 0x202F:
        return True
    return False

In [9]:
# whitespace가 2개인 경우를 처리해야 함

string1 = '1839년 파우스트을 읽었다.'
string2 = '1839년  파우스트을 읽었다.'
string1[6:10], string2[7:11]

('파우스트', '파우스트')

- 위와 같이 whitespace가 2개인 경우를 처리하려면 다르게 처리될 것입니다.

In [10]:
word_tokens = []
char_to_word = []
prev_is_whitespace = True

# 첫번째 문장 (string1) : 띄어쓰기 1칸 
for c in string1:
    if _is_whitespace(c):
        prev_is_whitespace = True
    else:
        if prev_is_whitespace:
            word_tokens.append(c)
        else:
            word_tokens[-1] += c
        prev_is_whitespace = False
    char_to_word.append(len(word_tokens)-1)
    print(f'\'{c}\':{word_tokens} : {char_to_word}')

'1':['1'] : [0]
'8':['18'] : [0, 0]
'3':['183'] : [0, 0, 0]
'9':['1839'] : [0, 0, 0, 0]
'년':['1839년'] : [0, 0, 0, 0, 0]
' ':['1839년'] : [0, 0, 0, 0, 0, 0]
'파':['1839년', '파'] : [0, 0, 0, 0, 0, 0, 1]
'우':['1839년', '파우'] : [0, 0, 0, 0, 0, 0, 1, 1]
'스':['1839년', '파우스'] : [0, 0, 0, 0, 0, 0, 1, 1, 1]
'트':['1839년', '파우스트'] : [0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
'을':['1839년', '파우스트을'] : [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
' ':['1839년', '파우스트을'] : [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
'읽':['1839년', '파우스트을', '읽'] : [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2]
'었':['1839년', '파우스트을', '읽었'] : [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2]
'다':['1839년', '파우스트을', '읽었다'] : [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2]
'.':['1839년', '파우스트을', '읽었다.'] : [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2]


In [11]:
word_tokens = []
char_to_word = []
prev_is_whitespace = True

# 두번째 문장 (string2) : 띄어쓰기 2칸 
for c in string2:
    if _is_whitespace(c):
        prev_is_whitespace = True
    else:
        if prev_is_whitespace:
            word_tokens.append(c)
        else:
            word_tokens[-1] += c
        prev_is_whitespace = False
    char_to_word.append(len(word_tokens)-1)
    print(f'\'{c}\':{word_tokens}:{char_to_word}')

'1':['1']:[0]
'8':['18']:[0, 0]
'3':['183']:[0, 0, 0]
'9':['1839']:[0, 0, 0, 0]
'년':['1839년']:[0, 0, 0, 0, 0]
' ':['1839년']:[0, 0, 0, 0, 0, 0]
' ':['1839년']:[0, 0, 0, 0, 0, 0, 0]
'파':['1839년', '파']:[0, 0, 0, 0, 0, 0, 0, 1]
'우':['1839년', '파우']:[0, 0, 0, 0, 0, 0, 0, 1, 1]
'스':['1839년', '파우스']:[0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
'트':['1839년', '파우스트']:[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
'을':['1839년', '파우스트을']:[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
' ':['1839년', '파우스트을']:[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
'읽':['1839년', '파우스트을', '읽']:[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2]
'었':['1839년', '파우스트을', '읽었']:[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2]
'다':['1839년', '파우스트을', '읽었다']:[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2]
'.':['1839년', '파우스트을', '읽었다.']:[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2]


- 위 기능을 함수로 만들겠습니다.

In [12]:
def _tokenize_whitespace(string):
    word_tokens = []
    char_to_word = []
    prev_is_whitespace = True
    
    for c in string:
        if _is_whitespace(c):
            prev_is_whitespace = True
        else:
            if prev_is_whitespace:
                word_tokens.append(c)
            else:
                word_tokens[-1] += c
            prev_is_whitespace = False
        char_to_word.append(len(word_tokens)-1)
    
    return word_tokens, char_to_word

- 함수 결과 확인

In [13]:
# 첫번째 문장(string1)에 대해 띄어쓰기 영역 정보를 표시
word_tokens, char_to_word = _tokenize_whitespace(string1)
for c, i in zip(list(string1), char_to_word):
    print(f'\'{c}\' : {i}')

word_tokens, char_to_word

'1' : 0
'8' : 0
'3' : 0
'9' : 0
'년' : 0
' ' : 0
'파' : 1
'우' : 1
'스' : 1
'트' : 1
'을' : 1
' ' : 1
'읽' : 2
'었' : 2
'다' : 2
'.' : 2


(['1839년', '파우스트을', '읽었다.'], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2])

In [14]:
# 두번째 문장(string2)에 대해 띄어쓰기 영역 정보를 표시
word_tokens, char_to_word = _tokenize_whitespace(string2)
for c, i in zip(list(string2), char_to_word):
    print(f'\'{c}\' : {i}')

word_tokens, char_to_word

'1' : 0
'8' : 0
'3' : 0
'9' : 0
'년' : 0
' ' : 0
' ' : 0
'파' : 1
'우' : 1
'스' : 1
'트' : 1
'을' : 1
' ' : 1
'읽' : 2
'었' : 2
'다' : 2
'.' : 2


(['1839년', '파우스트을', '읽었다.'],
 [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2])

## 0-2. 전처리 - Tokenize by Vocab
- Subword Segmentation 방식 
- 한국어 koNLPy의 형태소분석기를 사용합니다. 

In [ ]:
vocab = spm.SentencePieceProcessor()
vocab.load(f'{model_dir}/ko_32000.model')

word

# 루브릭 평가
---
1. **BERT pretrained model을 활용한 KorQuAD 모델이 정상적으로 학습이 진행되었다.**   

    - KorQuAD 모델의 validation accuracy가 안정적으로 증가하였다.


2. **KorQuAD Inference 결과가 원래의 정답과 비교하여 유사하게 나오는 것을 확인하였다.**

    - 평가셋에 대해 모델 추론 결과와 실제 정답의 유사성이 확인되었다.
     
     
3. **pretrained model 활용이 효과적임을 실험을 통해 확인하였다.** 

    - pretrained model을 사용하지 않았을 때 대비 학습경과의 차이를 시각화를 통해 확인하였다.

    
    
# 회고
---
## 어려웠던 점 
- 변수명을 잘못 입력한 부분 외에는 크게 어려운 점이 없었다. 

## 시도한 것들
- 최대한 효율적으로 결과를 비교하기 위해 함수를 사용해서 3가지 모델을 비교했다. 

## 다짐
- 특정 task를 해결하고자 할 때, 사용할 수 있는 방법론에 대해 평가하는 법을 익힐 수 있었다. 무작정 여러가지를 써보거나, 가장 이용률이 높은 것을 사용하는 것보다 해당 task에 가장 필요한 부분이 어떤 것인지를 정하고 그 기준에 대해서 평가하면 더 유의미한 실험결과를 도출할 수 있을 것이다.